In [1]:
from pathlib import Path

import ee
import geopandas as gpd
import pandas as pd
from geemap import gdf_to_ee

from fwi_predict.geo.ee import monitor_task, get_sentinel2_values_at_feature
from fwi_predict.gcs import download_files

In [2]:
ee.Initialize(project='fwi-water-quality-sensing')

In [3]:
predict_ds = gpd.read_file("../data/clean/jan_mar_25_w_metadata.geojson")

In [4]:
gcs_fp = Path('train/sentinel2/jan_mar_25_w_metadata').as_posix()

In [5]:
def create_sentinel2_feature_set(ds: gpd.GeoDataFrame,
                                 filename_prefix: str,
                                 bucket: str = 'fwi-predict',
																 back_days: int = 20):
	
	small_df = ds[['sample_idx', 'sample_dt', 'geometry']]
	ds_ee = gdf_to_ee(small_df, date='sample_dt', date_format="yyyy-MM-dd'T'HH:mm:ssZ")
	# ds_ee = ds_ee.map(lambda)
	fc = ds_ee.map(lambda f: get_sentinel2_values_at_feature(f, back_days=back_days))

	task = ee.batch.Export.table.toCloudStorage(
		collection=fc,
		description='mwm_sentinel2',
		bucket='fwi-predict',
		fileNamePrefix=gcs_fp,
		fileFormat='CSV'
	)
	task.start()

	task_success = monitor_task(task)

	if not task_success:
		print("Data export failed. Please consult GEE task manager for information.")
		return None
	
	download_files(bucket='fwi-predict',
                 file_glob=gcs_fp + '.csv',
                 download_dir='../data/gcs')
	
	print(f"Saving Sentinel-2 features to: ../data/gcs/{gcs_fp}.csv")


In [6]:
create_sentinel2_feature_set(predict_ds, filename_prefix=gcs_fp)

Task completed.
Saving Sentinel-2 features to: ../data/gcs/train/sentinel2/jan_mar_25_w_metadata.csv


In [7]:
sentinel_features = pd.read_csv("../data/gcs/train/sentinel2/jan_mar_25_w_metadata.csv").drop(columns=['system:index', '.geo'])

In [8]:
sentinel_features

,AOT,B1,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,B9,MSK_CLDPRB,SCL,WVP,hours_from_measurement,ndvi,ndwi,sample_idx
0,0.324,1.5280,0.6550,0.5697,1.1512,1.0499,1.0103,1.1048,1.0341,1.0015,0.8974,0.9676,1.6888,1.0,9,2.565,213.070316,-0.059181,0.078314,8518
1,0.324,1.5280,0.6550,0.5697,1.1512,1.0499,1.0103,1.1048,1.0341,1.0015,0.8974,0.9676,1.6888,1.0,9,2.565,220.903649,-0.059181,0.078314,8519
2,0.582,0.0447,0.0302,0.0143,0.0499,0.0689,0.0431,0.0746,0.0805,0.0928,0.0855,0.0743,0.1497,0.0,7,2.460,44.429662,0.329705,-0.107513,8806
3,0.582,0.0447,0.0302,0.0143,0.0499,0.0689,0.0431,0.0746,0.0805,0.0928,0.0855,0.0743,0.1497,0.0,7,2.460,53.212996,0.329705,-0.107513,8807
4,0.402,0.0131,0.0185,0.0108,0.0216,0.0518,0.0242,0.0579,0.0229,0.0208,0.0168,0.0041,0.1398,0.0,7,2.555,44.730508,-0.180488,0.510204,9122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,0.192,0.0476,0.0211,0.0137,0.0462,0.0556,0.0367,0.0643,0.0434,0.0458,0.0378,0.0297,0.0794,0.0,6,2.451,20.928910,0.014765,0.190578,8906
639,0.192,0.0476,0.0211,0.0137,0.0462,0.0556,0.0367,0.0643,0.0434,0.0458,0.0378,0.0297,0.0794,0.0,6,2.451,29.328910,0.014765,0.190578,8909
640,0.162,0.0448,0.0141,0.0103,0.0411,0.0524,0.0322,0.0527,0.0336,0.0322,0.0287,0.0244,0.0669,0.0,6,2.042,101.229481,-0.057471,0.292232,8765
641,0.126,0.0425,0.0210,0.0155,0.0320,0.0472,0.0249,0.0553,0.0348,0.0340,0.0286,0.0242,0.0803,0.0,6,2.540,115.490239,0.069159,0.245383,9211
